In [10]:
import tkinter as tk

In [11]:
class Estado:
    def __init__(self, fila, columna, costo, padre):
        self.fila = fila
        self.columna = columna
        self.costo = costo
        self.padre = padre

    def __lt__(self, other):
        # Comparación basada en el costo total (costo acumulado + costo estimado)
        return (self.costo) < (other.costo)

In [12]:
board_size = 8

# Inicializar los colores de las casillas
white_color = 'white'
black_color = 'black'
default_color = 'lightblue'
board = [[' ' for _ in range(board_size)] for _ in range(board_size)]
board[3][3] = board[4][4] = 'W'
board[3][4] = board[4][3] = 'B'
print(board)

[[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', 'W', 'B', ' ', ' ', ' '], [' ', ' ', ' ', 'B', 'W', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]


In [13]:
points_direction = {
    0: 'N',
    1: 'NE',
    2: 'E',
    3: 'SE',
    4: 'S',
    5: 'SO',
    6: 'O',
    7: 'NO'
}

directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def check_mov_direction(row, col,player):
    pos=0
    direction_check=[]
    check =False
    if player == "black":
        openent= 'W'
    else:
        openent= 'B'
    for dr, dc in directions:
        if(board[row+dr][col+dc]==openent):
            direction_check.append(points_direction[pos])
            
            check= True
        pos+=1   
    print(direction_check)    
    return direction_check,check

def verify_limiter(x,y):
    return (x>-1 or y>-1 or x<8 or y <8)

def check_mov(row, col,dir, player):
    coor_repaint=[]
    coor_repaint.append((row, col)) 
    if player == "black":
        openent= 'W'
        player = 'B'
    else:
        player = 'W'
        openent= 'B'
    y,x=row, col
    print("La direccion",dir, "filas y col",row, col)
    if dir == 'N':
        y-=1
        while board[y][x] == openent:
            coor_repaint.append((y,x)) 
            if verify_limiter(x,y-1):
                y-=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y-1][last_x] != player :
            coor_repaint=[]
    elif dir == 'NE':
        y-=1
        x+=1
        while  board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x+1,y-1):
                y-=1
                x+=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y-1][last_x+1] != player :
            coor_repaint=[]
    elif dir == 'E':
        x+=1 
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x+1,y):
                x+=1 
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y][last_x+1] != player :
            coor_repaint=[]
    elif dir == 'SE':
        y+=1
        x+=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x+1,y+1):
                y+=1
                x+=1 
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y+1][last_x+1] != player :
            coor_repaint=[]
    elif dir == 'S':
        y+=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            print("la ultima corr agregada", coor_repaint)
            if verify_limiter(x,y+1):
                y+=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        
        if board[last_y+1][last_x] != player :
            coor_repaint=[]
    elif dir == 'SO':
        y+=1
        x-=1 
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x-1,y+1):
                y+=1
                x-=1 
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y+1][last_x-1] != player:
            coor_repaint=[]
    elif dir == 'O':
        x-=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x-1,y):
                x-=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y][last_x-1] != player :
            coor_repaint=[]
    elif dir == 'NO':
        y-=1
        x-=1
        while board[y][x] == openent :
            coor_repaint.append((y,x)) 
            if verify_limiter(x-1,y-1):
                y-=1
                x-=1
            else:
                break
        last_y,last_x= coor_repaint[-1]
        if board[last_y-1][last_x-1] != player :
            coor_repaint=[]
    else:
        pass 
    print("LAs coor a repintar son", coor_repaint)       
    return coor_repaint

def coor_to_paint(row, col,direction_check, player):
    result=[]
    for dir in direction_check:
        res=check_mov(row, col,dir, player)
        if res != [] :
            #pintar todas las posibilidades
            result.append(res)
    return result

In [14]:


# Tamaño del tablero (8x8)

# Función para crear una casilla en el tablero
def crear_casilla(tablero, row, col, color=default_color):
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)
    button.grid(row=row + 1, column=col + 1)
    return button

def cambiar_color():
    global current_color
    current_color = 'W' if current_color == 'B' else 'B'
    label_color.config(text=f'Color actual: {current_color}')

def imprimir_coordenadas(row, col):
    print(f"Posición seleccionada: ({row}, {col})")
# Función para pintar una casilla
def pintar_casilla(row, col):
    if board[row][col] == ' ':
        color = white_color if current_color == 'W' else black_color
        direction_check,check= check_mov_direction(row, col, color)
        print("ES el turno de",color)
        if check:
            coors_repaints= coor_to_paint(row, col,direction_check, color)
            print("lo que llega", coors_repaints)
            for coor_repaint in coors_repaints:
                for y,x in coor_repaint:
                    board[y][x] = current_color
                    crear_casilla(tablero, y, x, color)
                imprimir_coordenadas(row, col)  
                cambiar_color()

root = tk.Tk()
root.title('Othello')

tablero = tk.Frame(root)
tablero.pack()

current_color = 'B'

label_color = tk.Label(root, text=f'Color actual: {current_color}')
label_color.pack()

def darw_map():
    for i in range(board_size):
        tk.Label(tablero, text=str(i + 1), width=3).grid(row=i + 1, column=0)
        tk.Label(tablero, text=chr(ord('A') + i), width=3).grid(row=0, column=i + 1)

    # Crear los botones del tablero
    for row in range(board_size):
        for col in range(board_size):
            if (row == 3 and col == 3) or (row == 4 and col == 4):
                crear_casilla(tablero, row, col, 'white')
            elif (row == 3 and col == 4) or (row == 4 and col == 3):
                crear_casilla(tablero, row, col, 'black')
            else:
                crear_casilla(tablero, row, col)

darw_map()
root.mainloop()


['S']
ES el turno de black
La direccion S filas y col 2 3
la ultima corr agregada [(2, 3), (3, 3)]
LAs coor a repintar son [(2, 3), (3, 3)]
lo que llega [[(2, 3), (3, 3)]]
Posición seleccionada: (2, 3)
['NE', 'E']
ES el turno de white
La direccion NE filas y col 4 2
LAs coor a repintar son []
La direccion E filas y col 4 2
LAs coor a repintar son [(4, 2), (4, 3)]
lo que llega [[(4, 2), (4, 3)]]
Posición seleccionada: (4, 2)
['N', 'NE', 'NO']
ES el turno de black
La direccion N filas y col 5 3
LAs coor a repintar son [(5, 3), (4, 3)]
La direccion NE filas y col 5 3
LAs coor a repintar son []
La direccion NO filas y col 5 3
LAs coor a repintar son []
lo que llega [[(5, 3), (4, 3)]]
Posición seleccionada: (5, 3)
['S', 'SO', 'O']
ES el turno de white
La direccion S filas y col 2 4
la ultima corr agregada [(2, 4), (3, 4)]
LAs coor a repintar son [(2, 4), (3, 4)]
La direccion SO filas y col 2 4
LAs coor a repintar son [(2, 4), (3, 3)]
La direccion O filas y col 2 4
LAs coor a repintar son []

In [15]:
points_direction[1]

'NE'

In [16]:
board[2][3]

'B'

In [17]:
from sys import maxsize

def min_max(state, player):
    if(player == "black"):
        v = []
        v.append(-maxsize)
        for action in actions(state):
            v.append(max_value(result(state, action)))
        return max(v)
    elif (player == "white"):
        v = []
        v.append(maxsize)
        for action in actions(state):
            v.append(min_value(result(state, action)))
        return min(v)        
    

def min_value(state):
    if(terminalState(state)):
        return utility(state)
    
    v = maxsize
    for action in actions(state):
        succesor =  result(state, action)
        eval_max = max_value(succesor)
        if(eval_max < v):
            v = eval_max
    return v

def max_value(state):
    if(terminalState(state)):
        return utility(state)
    
    v = -maxsize
    for action in actions(state):
        succesor =  result(state, action)
        eval_min = min_value(succesor)
        if(eval_max > v):
            v = eval_min
    return v

### CON PODA

In [18]:

def min_value(state, alpha, beta):
    if(terminalState(state)):
        return utility(state)
    v = -maxsize
    for action in actions(state):
        if(alpha >= beta):continue
        succesor =  result(state, action)
        eval_max = max_value(succesor, alpha, beta)
        if(eval_max < v):
            v = eval_max
            if((beta > v)):
                beta = v
    return v



def max_value(state, alpha, beta):
    if(terminalState(state)):
        return utility(state)
    v = -maxsize
    for action in actions(state):
        if(alpha >= beta):continue
        succesor =  result(state, action)
        eval_min = min_value(succesor, alpha, beta)
        if(eval_min > v):
            v = eval_min
            if((v > alpha)):
                alpha = v
    return v